Nesse projeto vamos desenvolver um modelo que possa analisar o comportamento do cliente e recomendar um dos planos mais recentes da Megaline: Smart ou Ultra. Para esta tarefa de classificação, irei desenvolver um modelo que escolherá o plano certo.
Meu primeiro passo foi importar todas as bibliotecas necessárias. 


In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from joblib import dump
import numpy as np
from collections import Counter


Nos próximos códigos realizei a análise dos dados da tabela 'users_behavior_upd'. Abri o arquivo usando pd.read_csv. 

In [2]:
df = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/moved_users_behavior.csv')

Resumo da tabela:

In [3]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


As principais informações da tabela. Nota-se que ela já está ajustada. As variáveis estão nos formatos certos e não há valores nulos. Podemos seguir para os próximos passos do projeto.   

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Abaixo a descrição dos dados de cada coluna: 

In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Abaixo descobri como estavam distribuidos os planos Smart e Ultra. Quase 70% das observações eram do plano Smart e apenas 30% do plano Ultra.

In [6]:
df['is_ultra'].value_counts() / len(df['is_ultra'])

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Dividi a tabela em target (objetivo) e features (características). Target representa a coluna 'is_ultra' e features as outras. 

In [7]:
target = df['is_ultra']
features = df.drop(['is_ultra'], axis=1)

In [8]:
features

,calls,minutes,messages,mb_used
0,40.0,311.90,83.0,19915.42
1,85.0,516.75,56.0,22696.96
2,77.0,467.66,86.0,21060.45
3,106.0,745.53,81.0,8437.39
4,66.0,418.74,1.0,14502.75
...,...,...,...,...
3209,122.0,910.98,20.0,35124.90
3210,25.0,190.36,0.0,3275.61
3211,97.0,634.44,70.0,13974.06
3212,64.0,462.32,90.0,31239.78


Para nosso modelo funcionar precisamos ter além de um conjunto para treinar o modelo, um conjunto para validar e depois testá-lo. Por isso, nos próximos códigos dividi os dados da seguinte forma: 60% conjunto de teste, 20% conjunto de validação e 20% conjunto de teste.

In [9]:
df_train, df_valid = train_test_split(df, test_size=0.20, random_state=12345)

In [10]:
df_train, df_test = train_test_split(df_train, test_size=0.25, random_state=12345)

Abaixo analisei como ficaram as novas tabelas: df_train, df_test e df_valid. Me certifiquei que a proporção de planos Smart e Ultra manteve-se similar à do conjunto original. 

In [11]:
df_train

,calls,minutes,messages,mb_used,is_ultra
2656,30.0,185.07,34.0,17166.53,0
823,42.0,290.69,77.0,21507.03,0
2566,41.0,289.83,15.0,22151.73,0
1451,45.0,333.49,50.0,17275.47,0
2953,43.0,300.39,69.0,17277.83,0
...,...,...,...,...,...
1043,106.0,796.79,23.0,42250.70,1
2132,18.0,117.80,0.0,10006.79,1
1642,87.0,583.02,1.0,11213.97,0
1495,63.0,408.68,63.0,24970.26,0


In [12]:
df_train['is_ultra'].value_counts() / len(df_train['is_ultra'])

0    0.694502
1    0.305498
Name: is_ultra, dtype: float64

In [13]:
df_train['is_ultra'].value_counts() 

0    1339
1     589
Name: is_ultra, dtype: int64

In [14]:
df_test

,calls,minutes,messages,mb_used,is_ultra
2699,71.0,512.53,27.0,15772.68,0
242,183.0,1247.04,150.0,29186.41,1
2854,34.0,246.06,31.0,8448.76,0
1638,63.0,468.66,0.0,11794.34,0
1632,4.0,19.85,28.0,13107.42,0
...,...,...,...,...,...
2551,83.0,539.32,33.0,20967.28,0
1261,80.0,552.33,23.0,18457.85,0
1658,111.0,899.74,0.0,26866.61,0
353,68.0,493.00,29.0,20021.73,0


In [15]:
df_test['is_ultra'].value_counts() / len(df_test['is_ultra'])

0    0.688958
1    0.311042
Name: is_ultra, dtype: float64

In [16]:
df_test['is_ultra'].value_counts()

0    443
1    200
Name: is_ultra, dtype: int64

In [17]:
df_valid

,calls,minutes,messages,mb_used,is_ultra
1415,82.0,507.89,88.0,17543.37,1
916,50.0,375.91,35.0,12388.40,0
1670,83.0,540.49,41.0,9127.74,0
686,79.0,562.99,19.0,25508.19,1
2951,78.0,531.29,20.0,9217.25,0
...,...,...,...,...,...
2061,66.0,478.48,0.0,16962.58,0
1510,40.0,334.24,91.0,11304.14,0
2215,62.0,436.68,52.0,12311.24,0
664,117.0,739.27,124.0,22818.56,1


In [18]:
df_valid['is_ultra'].value_counts() / len(df_valid['is_ultra'])

0    0.695179
1    0.304821
Name: is_ultra, dtype: float64

In [19]:
df_valid['is_ultra'].value_counts()

0    447
1    196
Name: is_ultra, dtype: int64

Assim como fiz anteriormente para a tabela toda, dividi o conjunto teste, validação e treinamento em target (objetivo) e features (características). Target representa a coluna 'is_ultra' e features as outras. 

In [20]:
target_train = df_train['is_ultra']
features_train = df_train.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)

In [21]:
features_valid

,calls,minutes,messages,mb_used
1415,82.0,507.89,88.0,17543.37
916,50.0,375.91,35.0,12388.40
1670,83.0,540.49,41.0,9127.74
686,79.0,562.99,19.0,25508.19
2951,78.0,531.29,20.0,9217.25
...,...,...,...,...
2061,66.0,478.48,0.0,16962.58
1510,40.0,334.24,91.0,11304.14
2215,62.0,436.68,52.0,12311.24
664,117.0,739.27,124.0,22818.56


Começamos a partir daqui com as investigações de qual modelo seria o ideal para esse estudo. Para isso, testei 3 modelos distintos: árvore de decisão, floresta aleatória e regressão logística. Esses foram os modelos apresentados quando estamos estudando variáveis categóricas. 

Árvore de Decisão

Primeiro modelo tentado foi árvore de decisão. Realizei um ciclo for alterando a profundidade máxima. Cheguei à conclusão que o a profundidade 5 era a ideal, pois maximizou a acurácia.

In [22]:
best_accuracy_tree = 0
best_max_depth_tree = 0

for i in (range(1,30)):
    model_tree = DecisionTreeClassifier(random_state=12345,max_depth=i)
    model_tree.fit(features_train, target_train)
    valid_predictions  = model_tree.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, valid_predictions)
    if accuracy_valid > best_accuracy_tree:
        best_accuracy_tree = accuracy_valid
        best_max_depth_tree = i
        
        
    
print("A acurácia do melhor modelo no conjunto de validação (max_depth = {}): {}".format(best_max_depth_tree, best_accuracy_tree))

A acurácia do melhor modelo no conjunto de validação (max_depth = 5): 0.7884914463452566


Após usar o conjunto de validação para validar o modelo. Abaixo usei o conjunto teste para testar o modelo. Nesse conjunto a acurácia diminuiu mas manteve-se acima 0,75 que foi o limite imposto pelo projeto.

In [23]:
model_tree = DecisionTreeClassifier(random_state=12345,max_depth=5)
model_tree.fit(features_train, target_train)
test_predictions_tree  = model_tree.predict(features_test)
accuracy_test = accuracy_score(target_test, test_predictions_tree)
print('A acurácia do melhor modelo de Árvore de decisão no conjunto de teste é:',accuracy_test ) 

A acurácia do melhor modelo de Árvore de decisão no conjunto de teste é: 0.7589424572317263


Também conferi como ficou a distribuição dos planos Smart e Ultra nas predições do modelo árvore de decisão nos conjuntos de validação e treinamento. Em ambos, o plano Smart representou aproximadamente 85% das observações. Participação maior da encontrada na tabela original que foi de 70%. 

In [25]:
ultra1 = test_predictions_tree.sum()/len(test_predictions_tree)
smart1 = (len(test_predictions_tree)-test_predictions_tree.sum())/len(test_predictions_tree)

In [26]:
print('Participação Ultra:',ultra1) 
print('Participação Smart:',smart1) 

Participação Ultra: 0.14152410575427682
Participação Smart: 0.8584758942457231


In [28]:
ultra2 = valid_predictions_tree.sum()/len(valid_predictions_tree)
smart2 = (len(valid_predictions_tree)-valid_predictions_tree.sum())/len(valid_predictions_tree)
print('Participação Ultra:',ultra2) 
print('Participação Smart:',smart2) 

Participação Ultra: 0.15863141524105753
Participação Smart: 0.8413685847589425


Floresta Aleatória

Segundo modelo tentado foi Floresta Aleatória. Realizei um ciclo for alterando o número de estimadores. Cheguei à conclusão de que 17 era o número de estimadores ideal, pois maximizou a acurácia. A acurácia de 0,7947 foi maior em relação a encontrada no modelo anterior. 

In [29]:
best_score_randforest = 0
best_est_randforest = 0
for est2 in range(1, 30): # escolha o intervalo para o hiperparâmetro
    model_randforest = RandomForestClassifier(random_state=54321, n_estimators=est2) # defina o número de árvores
    model_randforest.fit(features_train,target_train) # use o conjunto de treinamento para treinar o modelo
    score_randforest = model_randforest.score(features_valid,target_valid) # calcule a acurácia no conjunto de validação
    if score_randforest > best_score_randforest:
        best_score_randforest = score_randforest # salve o melhor resultado da acurácia no conjunto de validação
        best_est_randforest = est2 # salve o número de estimadores que corresponde ao melhor resultado de acurácia

print("A acurácia do melhor modelo no conjunto de validação (n_estimators = {}): {}".format(best_est_randforest, best_score_randforest))



A acurácia do melhor modelo no conjunto de validação (n_estimators = 17): 0.7947122861586314


Após usar o conjunto de validação para validar o modelo. Abaixo usei o conjunto teste para testar o modelo. Nesse conjunto a acurácia diminuiu muito pouco e manteve-se acima 0,79.

In [30]:
model_randforest = RandomForestClassifier(random_state=54321, n_estimators=17)
model_randforest.fit(features_train,target_train)
score_randforest = model_randforest.score(features_test,target_test)
print("A acurácia do melhor modelo no conjunto de teste (n_estimators = 17): {}".format(score_randforest))

A acurácia do melhor modelo no conjunto de teste (n_estimators = 17): 0.7900466562986003


Também conferi novamente como ficou a distribuição dos planos Smart e Ultra nas predições do modelo Floresta Aleatória nos conjuntos de validação e treinamento. Em ambos, o plano Smart representou aproximadamente 75% das observações. Participação maior da encontrada na tabela original que foi de 70%, mas mais próxima em relação ao modelo anterior. 

In [32]:
ultra3 = test_predictions_randforest.sum()/len(test_predictions_randforest)
smart3 = (len(test_predictions_randforest)-test_predictions_randforest.sum())/len(test_predictions_randforest)
print('Participação Ultra:',ultra3) 
print('Participação Smart:',smart3) 

Participação Ultra: 0.2379471228615863
Participação Smart: 0.7620528771384136


In [34]:
ultra4 = valid_predictions_randforest.sum()/len(valid_predictions_randforest)
smart4 = (len(valid_predictions_randforest)-valid_predictions_randforest.sum())/len(valid_predictions_randforest)
print('Participação Ultra:',ultra4) 
print('Participação Smart:',smart4) 

Participação Ultra: 0.24572317262830481
Participação Smart: 0.7542768273716952


Regressão Logística

O último modelo tentado foi Regressão Logística. Fiz algumas tentativas trocando o valor de random_state mas a acurácia desse modelo manteve-se sempre próxima a 74%. Logo, abaixo dos 0,75 requisitados pelo projeto. Esse foi certamente o pior modelo dos 3 tentados.     

In [35]:
model_log_reg = LogisticRegression(random_state=54321, solver='liblinear') # inicialize o construtor de regressão logística com os parâmetros random_state=54321 e solver='liblinear'
model_log_reg.fit(features_train,target_train) # use o conjunto de treinamento para treinar o modelo
score_train_log_reg = model_log_reg.score(features_train,target_train) # calcule a acurácia no conjunto de treinamento
score_valid_log_reg = model_log_reg.score(features_valid,target_valid) # calcule a acurácia no conjunto de validação

print("Acurácia do modelo de regressão logística no conjunto de treinamento:", score_train_log_reg)
print("Acurácia do modelo de regressão logística no conjunto de validação:", score_valid_log_reg)

Acurácia do modelo de regressão logística no conjunto de treinamento: 0.7422199170124482
Acurácia do modelo de regressão logística no conjunto de validação: 0.7511664074650077


Conferi como ficou a distribuição dos planos Smart e Ultra nas predições do modelo Regressão Logística nos conjuntos de validação e treinamento. Em ambos, o plano Smart representou aproximadamente 94% das observações. Participação muito maior da encontrada na tabela original que foi de 70%. Esse resultado explicou o porquê de a acurácia do modelo ter ficado mais próxima a 70%. Ele basicamente assumiu que quase todos os planos eram Smart. Como originalmente 70% das observações eram desse plano, a acurácia ficou mais perto de 0,7. 

In [37]:
ultra5 = test_predictions_log_reg.sum()/len(test_predictions_log_reg)
smart5 = (len(test_predictions_log_reg)-test_predictions_log_reg.sum())/len(test_predictions_log_reg)
print('Participação Ultra:',ultra5) 
print('Participação Smart:',smart5) 

Participação Ultra: 0.05287713841368585
Participação Smart: 0.9471228615863142


In [39]:
ultra6 = valid_predictions_log_reg.sum()/len(valid_predictions_log_reg)
smart6 = (len(valid_predictions_log_reg)-valid_predictions_log_reg.sum())/len(valid_predictions_log_reg)
print('Participação Ultra:',ultra6) 
print('Participação Smart:',smart6) 

Participação Ultra: 0.06531881804043546
Participação Smart: 0.9346811819595645


Após validar e testar 3 modelos diferentes e alterar os hiperparâmetros dentro deles, cheguei à conclusão de que o melhor modelo para escolher o plano certo foi o modelo Floresta Aleatória com 17 estimadores. Mesmo esse modelo tendo uma performance computacional pior aos outros, nesse caso não senti nenhuma demora em rodar o código, logo esse quesito não foi um problema. O modelo floresta Aleatória teve a maior acurácia tanto no conjunto de validação como no conjunto de teste em relação aos demais. Também, a proporção de planos Smart e Ultra ficou mais próxima ao dos dados originais. Por esses motivos, elegi esse modelo como o melhor para escolher o plano certo de acordo com as características apresentadas.